In [1]:
import os
import re
import csv
import uuid
import glob
import shutil

In [2]:
# Variables
source_dir = "source_files/"
source_metadata_file = source_dir + "papers.csv"

table_output_dir = "extracted_tables/"
table_code_dir = table_output_dir + "table_code/"
table_result_file = table_output_dir + "tables.csv"

figure_output_dir = "extracted_figures/"
figure_result_file = figure_output_dir + "figures.csv"

possible_extensions = [".pdf", ".png", ".jpg", ".jpeg", ".eps"]

os.makedirs(table_output_dir, exist_ok=True)
os.makedirs(table_code_dir, exist_ok=True)
os.makedirs(figure_output_dir, exist_ok=True)

if os.path.isfile(table_result_file) == False:
    open(table_result_file, "w").close()
    
if os.path.isfile(figure_result_file) == False:
    open(figure_result_file, "w").close()

In [22]:
# Reset directories and csv files
# WARNING: This deletes all collected tables and figures
files = glob.glob(table_code_dir + "*")
for f in files:
    os.remove(f)
    
files = glob.glob(figure_output_dir + "*")
for f in files:
    os.remove(f)
open(figure_result_file, "w").close()

if os.path.isfile(table_result_file):
    os.remove(table_result_file)
    open(table_result_file, "w").close()

In [23]:
"""
Extracting tables and figures from the downloaded papers in the source directory
Start - index of the first paper to be used (beginning from 0)
End - index of the last paper to be used
"""
def extract_data_from_papers(start, end):
    csvfile_metadata = open(source_metadata_file, "r")
    spamreader = csv.reader(csvfile_metadata, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)

    for index, row in enumerate(spamreader):
        if index < start:
            continue
        if index > end:
            break
        
        paper_id = row[0]
        paper_path = source_dir + paper_id
        if os.path.isdir(paper_path):
            print("\n\n" + paper_id)
            tex_files = [os.path.join(root, file)
                 for root, dirs, files in os.walk(paper_path)
                 for file in files if file.endswith(".tex") and file.startswith("FR_") == False]
            print(tex_files)

            complete_tex = ""
            for file in tex_files:
                try:
                    f = open(file, "r", encoding="utf8")
                    complete_tex += f.read()
                    f.close()
                except UnicodeDecodeError as e:
                    print("UnicodeDecodeError occurred. File could not be loaded.")
                    continue

            found_document_header = re.search(r"\\documentclass\[.*?\\begin\{document\}", complete_tex, re.DOTALL)
            if found_document_header is None:
                print("Document header could not be identified.")
                continue

            found_tables = re.findall(r"\\begin\{table\*?\}.*?\\end\{table\*?\}", complete_tex, re.DOTALL)
            found_figures = re.findall(r"\\begin\{figure\*?\}.*?\\end\{figure\*?\}", complete_tex, re.DOTALL)
            counter_table = 0
            counter_figure = 0

            for table in found_tables:
                #r"\\caption\{(.*?)}"
                #caption_match = re.search(r"\\caption\{(([^{}]*(\{[^{}]*\})?[^{}]*)+)\}", table)

                label_match = re.search(r"\\label\{(.*?)\}", table)
                caption_match = re.search(r"\\caption\{(.*?)}", table)
                
                if label_match and caption_match:
                    
                    # Skipping if no paragraph mention was found
                    found_label = label_match.group(1)
                    found_paragraphs = re.findall(fr".*\\ref\{{{re.escape(found_label)}}}.*", complete_tex)
                    if len(found_paragraphs) == 0:
                        continue
                        
                    paragraph_mentions = ""
                    for paragraph in found_paragraphs:
                        paragraph_mentions += paragraph
      
                    found_caption = caption_match.group(1)
                    counter_table += 1
                    table_id = paper_id + "_TAB_" + str(counter_table)

                    # Store meta information in csv file
                    with open(table_result_file, 'a', newline='', encoding="utf-8") as csvfile_table:
                        csvfile_table.write(table_id + ";" + paper_id + ";https://arxiv.org/abs/" + row[0] + ";" + found_caption + ";" + paragraph_mentions + "\n")

                    # Store as txt file in own data collection
                    table_file_path = table_code_dir + table_id + ".txt"
                    f = open(table_file_path, "w", encoding="utf-8")
                    f.write(table)
                    f.close()

                    # Store as tex file in folder of paper
                    table_file_path = paper_path + "/FR_TAB_" + table_id + ".tex"
                    table_tex_code = found_document_header.group(0) + "\n\pagenumbering{gobble}\n" + table + "\n\end{document}"
                    f = open(table_file_path, "w", encoding="utf-8")
                    f.write(table_tex_code)
                    f.close()

            for figure in found_figures:
                found_graphics = re.findall(r"\\includegraphics(\[.*?\])*\{(.*?)\}", figure)
                #caption_match = re.search(r"\\caption\{(([^{}]*(\{[^{}]*\})?[^{}]*)+)\}", figure)
                caption_match = re.search(r"\\caption\{(.*?)}", figure)
                label_match = re.search(r"\\label\{(.*?)\}", figure)
                
                # Skipping multi figures
                if len(found_graphics) != 1:
                    continue
                
                # Caption and label need to be found
                if caption_match and label_match:                  
                    
                    # Skipping if no paragraph mention was found
                    found_label = label_match.group(1)
                    found_paragraphs = re.findall(fr".*\\ref\{{{re.escape(found_label)}}}.*", complete_tex)
                    if len(found_paragraphs) == 0:
                        continue
                        
                    paragraph_mentions = ""
                    for paragraph in found_paragraphs:
                        paragraph_mentions += paragraph
                    paragraph_mentions.replace(";", ",")
                    
                    found_caption = caption_match.group(1)
                    graphic = found_graphics[0]

                    graphic_path = graphic[1]
                    file_type = os.path.splitext(graphic_path)[-1]
                    if os.path.isfile(paper_path+"/"+graphic_path):
                        graphic_path = paper_path+"/"+graphic_path
                    else:
                        unknown_extension = True
                        for ext in possible_extensions:
                            possible_path = paper_path+"/"+graphic_path + ext
                            if os.path.isfile(possible_path):
                                graphic_path = possible_path
                                file_type = ext
                                unknown_extension = False
                                break
                        if unknown_extension:
                            continue

                    try:
                        counter_figure += 1
                        figure_id = paper_id + "_FIG_" + str(counter_figure)
                        shutil.copy(graphic_path, figure_output_dir+figure_id+file_type)
                        with open(figure_result_file, 'a', newline='', encoding="utf-8") as csvfile_figure:
                            csvfile_figure.write(figure_id + ";" + row[0] + ";https://arxiv.org/abs/" + row[0] + ";" + found_caption + ";" + paragraph_mentions + "\n")
                    except FileNotFoundError as e:
                        print(f"File not found: {graphic_path} - {e}")
                    except Exception as e:
                        print(f"An error occurred: {e}")

            print(f"{counter_table}/{len(found_tables)} tables collected.")
            print(f"{counter_figure}/{len(found_figures)} figures collected.")

    csvfile_metadata.close()

In [24]:
extract_data_from_papers(0,651)



2303.11192
['source_files/2303.11192\\appendix.tex', 'source_files/2303.11192\\main.tex', 'source_files/2303.11192\\misc\\macros.tex']
0/4 tables collected.
0/9 figures collected.


2304.09510
['source_files/2304.09510\\ms.tex', 'source_files/2304.09510\\src\\abstract.tex', 'source_files/2304.09510\\src\\background.tex', 'source_files/2304.09510\\src\\conclusion.tex', 'source_files/2304.09510\\src\\discussion.tex', 'source_files/2304.09510\\src\\framework.tex', 'source_files/2304.09510\\src\\intro.tex', 'source_files/2304.09510\\src\\machines.tex', 'source_files/2304.09510\\src\\related.tex', 'source_files/2304.09510\\src\\simulator.tex']
0/0 tables collected.
0/18 figures collected.


2305.01851
['source_files/2305.01851\\arXiv.tex']
Document header could not be identified.


2307.16446
['source_files/2307.16446\\macros.tex', 'source_files/2307.16446\\main.tex']
1/1 tables collected.
5/6 figures collected.


2309.07322
['source_files/2309.07322\\eccv_supp.tex', 'source_files/2309.07

23/28 tables collected.
11/19 figures collected.


2403.07073
['source_files/2403.07073\\main.tex']
0/0 tables collected.
7/7 figures collected.


2403.09941
['source_files/2403.09941\\discontinuous-drift-aw.tex', 'source_files/2403.09941\\img\\example-epsilon.tex', 'source_files/2403.09941\\img\\example-limit.tex', 'source_files/2403.09941\\img\\second-order-1a.tex', 'source_files/2403.09941\\img\\second-order-1b.tex', 'source_files/2403.09941\\img\\second-order-2a.tex', 'source_files/2403.09941\\img\\second-order-2b.tex']
0/0 tables collected.
0/5 figures collected.


2403.11787
['source_files/2403.11787\\main.tex']
9/9 tables collected.
0/3 figures collected.


2403.14484
['source_files/2403.14484\\conference_101719.tex']
3/3 tables collected.
7/9 figures collected.


2403.16877
['source_files/2403.16877\\root.tex']
0/3 tables collected.
0/5 figures collected.


2403.18501
['source_files/2403.18501\\main.tex', 'source_files/2403.18501\\sec\\1_intro.tex', 'source_files/2403.18501\\se

Document header could not be identified.


2406.00010
['source_files/2406.00010\\EnterpriseEM.tex']
Document header could not be identified.


2406.10069
['source_files/2406.10069\\paper.tex']
4/4 tables collected.
3/3 figures collected.


2406.10858
['source_files/2406.10858\\acl_latex.tex']
5/6 tables collected.
2/5 figures collected.


2406.18445
['source_files/2406.18445\\conclusions.tex', 'source_files/2406.18445\\framework.tex', 'source_files/2406.18445\\heterojunction.tex', 'source_files/2406.18445\\introduction.tex', 'source_files/2406.18445\\main.tex', 'source_files/2406.18445\\smartpixel.tex', 'source_files/2406.18445\\svm.tex']
0/0 tables collected.
14/14 figures collected.


2406.18745
['source_files/2406.18745\\aaai25.tex']
0/9 tables collected.
0/7 figures collected.


2407.00518
['source_files/2407.00518\\main.tex']
0/2 tables collected.
0/3 figures collected.


2407.01733
['source_files/2407.01733\\main.tex', 'source_files/2407.01733\\SI_movie.tex']
0/0 tables collected

5/7 tables collected.
7/10 figures collected.


2409.16688
['source_files/2409.16688\\commands.tex', 'source_files/2409.16688\\main.tex', 'source_files/2409.16688\\packages.tex']
2/2 tables collected.
0/0 figures collected.


2409.17073
['source_files/2409.17073\\main.tex', 'source_files/2409.17073\\sections\\01abstract.tex', 'source_files/2409.17073\\sections\\02introduction.tex', 'source_files/2409.17073\\sections\\03related_work.tex', 'source_files/2409.17073\\sections\\04method.tex', 'source_files/2409.17073\\sections\\05dataset.tex', 'source_files/2409.17073\\sections\\06evaluation.tex', 'source_files/2409.17073\\sections\\07results.tex', 'source_files/2409.17073\\sections\\08conclusion.tex', 'source_files/2409.17073\\sections\\09limitations.tex']
10/10 tables collected.
3/5 figures collected.


2409.17545
['source_files/2409.17545\\main.tex']
1/1 tables collected.
4/5 figures collected.


2409.17670
['source_files/2409.17670\\manuscript.tex']
Document header could not be identifi

0/0 tables collected.
0/10 figures collected.


2409.18257
['source_files/2409.18257\\main.tex']
0/0 tables collected.
3/3 figures collected.


2409.18263
['source_files/2409.18263\\acl_main.tex']
0/11 tables collected.
0/4 figures collected.


2409.18267
['source_files/2409.18267\\main.tex']
Document header could not be identified.


2409.18268
['source_files/2409.18268\\arxive-Sep26.tex']
1/2 tables collected.
4/5 figures collected.


2409.18272
['source_files/2409.18272\\01_Introduction.tex', 'source_files/2409.18272\\02_1_DampingInMBS.tex', 'source_files/2409.18272\\02_Framework_Methods.tex', 'source_files/2409.18272\\03_MultibodySystems.tex', 'source_files/2409.18272\\05_Conclusion.tex', 'source_files/2409.18272\\Appendix.tex', 'source_files/2409.18272\\Commands.tex', 'source_files/2409.18272\\SLIDE_DeepLearningForMBS.tex']
0/4 tables collected.
0/27 figures collected.


2409.18273
['source_files/2409.18273\\root.tex']
1/2 tables collected.
2/6 figures collected.


2409.18275
['so

...

4/5 tables collected.
2/6 figures collected.


2409.17427
['source_files/2409.17427\\main.tex', 'source_files/2409.17427\\Text\\1Introduction.tex', 'source_files/2409.17427\\Text\\2RelatedWork.tex', 'source_files/2409.17427\\Text\\3Methodology.tex', 'source_files/2409.17427\\Text\\4Evaluation.tex', 'source_files/2409.17427\\Text\\5Results.tex', 'source_files/2409.17427\\Text\\6Discussion.tex', 'source_files/2409.17427\\Text\\7Conclusions.tex', 'source_files/2409.17427\\Text\\8Acknowledgements.tex', 'source_files/2409.17427\\Text\\introduction_old.tex']
5/5 tables collected.
1/3 figures collected.


2409.17431
['source_files/2409.17431\\arxiv_0923_stable.tex', 'source_files/2409.17431\\arxiv_template.tex', 'source_files/2409.17431\\diff.tex', 'source_files/2409.17431\\math_commands.tex']
Document header could not be identified.


2409.17434
['source_files/2409.17434\\Harnessing_the_Potential_of_Gen-AI_Coding_Assistants_in_Public_Sector_Software_Development.tex']
0/2 tables collected.
0/

2/13 tables collected.
4/9 figures collected.


2409.17578
['source_files/2409.17578\\main.tex', 'source_files/2409.17578\\sections\\1-introduction.tex', 'source_files/2409.17578\\sections\\2-literature_review.tex', 'source_files/2409.17578\\sections\\3-methods.tex', 'source_files/2409.17578\\sections\\4-findings.tex', 'source_files/2409.17578\\sections\\5-discussion.tex', 'source_files/2409.17578\\sections\\6-design_implications.tex', 'source_files/2409.17578\\sections\\7-conclusion.tex', 'source_files/2409.17578\\sections\\appendices.tex', 'source_files/2409.17578\\sections\\participants_summary.tex', 'source_files/2409.17578\\sections\\participants_table.tex', 'source_files/2409.17578\\sections\\table1.tex', 'source_files/2409.17578\\sections\\table2.tex', 'source_files/2409.17578\\sections\\table3.tex']
3/4 tables collected.
0/2 figures collected.


2409.17580
['source_files/2409.17580\\accuracy.tex', 'source_files/2409.17580\\commands-packages.tex', 'source_files/2409.17580\\datas

2/2 tables collected.
2/8 figures collected.


2409.17669
['source_files/2409.17669\\SEIR_opinion_contacts_arxiv.tex']
0/0 tables collected.
0/17 figures collected.


2409.17673
['source_files/2409.17673\\appendix.tex', 'source_files/2409.17673\\main.tex', 'source_files/2409.17673\\main_body.tex']
7/8 tables collected.
0/2 figures collected.


2409.17677
['source_files/2409.17677\\iclr2025_conference.tex', 'source_files/2409.17677\\math_commands.tex']
0/1 tables collected.
0/0 figures collected.


2409.17683
['source_files/2409.17683\\main.tex']
Document header could not be identified.


2409.17686
['source_files/2409.17686\\main.tex', 'source_files/2409.17686\\sections\\1_introduction.tex', 'source_files/2409.17686\\sections\\2_related_work.tex', 'source_files/2409.17686\\sections\\3_method.tex', 'source_files/2409.17686\\sections\\4_experiments.tex', 'source_files/2409.17686\\sections\\5_conclusion.tex', 'source_files/2409.17686\\sections\\6_appendix.tex', 'source_files/2409.17686\\s

0/0 tables collected.
7/7 figures collected.


2409.17858
['source_files/2409.17858\\iclr2024_conference.tex', 'source_files/2409.17858\\math_commands.tex']
Document header could not be identified.


2409.17876
['source_files/2409.17876\\manuscript.tex']
4/4 tables collected.
1/1 figures collected.


2409.17881
['source_files/2409.17881\\TXT_TII_24_0715.tex']
4/5 tables collected.
3/7 figures collected.


2409.17885
['source_files/2409.17885\\Conclusion.tex', 'source_files/2409.17885\\Discussion.tex', 'source_files/2409.17885\\Evaluation.tex', 'source_files/2409.17885\\Introduction.tex', 'source_files/2409.17885\\main.tex', 'source_files/2409.17885\\Methodology.tex', 'source_files/2409.17885\\Related_work.tex', 'source_files/2409.17885\\Threats.tex']
0/0 tables collected.
6/6 figures collected.


2409.17886
['source_files/2409.17886\\main.tex', 'source_files/2409.17886\\src\\1_introduction.tex', 'source_files/2409.17886\\src\\2_related.tex', 'source_files/2409.17886\\src\\3_method.tex'

2/2 tables collected.
1/2 figures collected.


2409.18047
['source_files/2409.18047\\root.tex']
0/0 tables collected.
5/7 figures collected.


2409.18062
['source_files/2409.18062\\bachelor_daniel_ketels.tex', 'source_files/2409.18062\\Institutsvorlage.tex']
6/6 tables collected.
0/24 figures collected.


2409.18063
['source_files/2409.18063\\appendix.tex', 'source_files/2409.18063\\background.tex', 'source_files/2409.18063\\eval.tex', 'source_files/2409.18063\\intro.tex', 'source_files/2409.18063\\lprf.tex', 'source_files/2409.18063\\main-pratenddefaultcategory.tex', 'source_files/2409.18063\\main.tex', 'source_files/2409.18063\\prelude.tex', 'source_files/2409.18063\\prf.tex', 'source_files/2409.18063\\related.tex']
0/2 tables collected.
0/2 figures collected.


2409.18073
['source_files/2409.18073\\macros.tex', 'source_files/2409.18073\\math.tex', 'source_files/2409.18073\\output.tex', 'source_files/2409.18073\\packages.tex', 'source_files/2409.18073\\arxiv-texts\\0-abstract.tex', '